# 1. Logistic Regression

(a) 
> (10 points) Do natural log transform of the PFOS variable in file
pfas.csv and store the results as a new variable log PFOS in the
data file. Standardize the variables x=[log PFOS, age, gender, BMI].

In [189]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as api
import statsmodels

In [24]:
df = pd.read_csv("pfas.csv")

In [25]:
def autoscale(df, variables_to_scale):
    """standardizes variables"""
    for variable in variables_to_scale:
        df[variable] = ( df[variable] - np.mean(df[variable]) ) / np.std(df[variable])

In [26]:
df["log_PFOS"] = np.log2(df["PFOS"])
standard_df = autoscale(df, ["log_PFOS", "age", "gender", "BMI"])

### Numerical Solution
(b) 
> (35 points) Use y=disease and the standardized x=[PFOS, age,
gender, BMI] to write and debug your own gradient descent algorithm for logistic regression. Your algorithm should export the
learned parameters in the θ vector. Note that you can modify the
gradient descent algorithm that you have written for the linear re-
gression algorithm to achieve logistic regression.

#### Initalize variables

In [166]:
Y = df["disease"].to_numpy()
X = df.loc[:,["log_PFOS", "age", "gender", "BMI"]].to_numpy()
## add intercept
X_with_intercept = np.insert( X, 0, np.array([1]*X.shape[0]), axis = 1 )



print("disease\n", Y)
print("     intercept|    PFOS   |   age     |  gender    |    BMI\n",X)



disease
 [0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 1
 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 0
 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 1 1 0
 0 1 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1
 0 1 1 0 1 0 0 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 0 1 1
 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1
 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 0 1 1 0 0
 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1
 0 0 1 0]
     intercept|    PFOS   |   age     |  gender    |    BMI
 [[ 0.90375231 -1.06321047 -1.06191317 -0.78453458]
 [-0.99778188  0.96375408 -1.06191317 -1.18985472]
 [-0.24350772  0.31512542  0.94169658 -0.5470809 ]
 ...
 [-0.24248557  0.396204    0.94169658  0.07829738]
 [-0.23195204  0.15296826  0.94169658  0.8139823 ]
 [ 0.29556023  0.80159691 -1.06191317  0.33858428]]


#### Sigmoid

$ g(B^Tx)= \frac{1}{1+e^{-(B^Tx)}}$

$B^Tx=B_0x_0+B_1x_1+...+B_nx_n$

$x_0=1$


$$
y = \begin{cases}
    \text{1 if} & B^Tx\text{ ≥ 0} \\
    \text{0 if} & B^Tx\text{< 0} \\
\end{cases}
$$


#### Cost Function

$i$ is a row $j$ is a column

$ J(B) = -\frac{1}{m} \sum_{i=1}^m [y_i ln(\frac{1}{1+e^{-(B^Tx)}}) + (1-y_i)ln(\frac{1}{1+e^{-(B^Tx)}})]$

$ B_j := B_j-α(\frac{∂J(B)}{∂B_j}) $

$ \frac{∂J(B)}{∂B_j} = \frac{1}{m} \sum [(\frac{1}{1+e^{-(B^Tx)}}) - y_i]x_{j,i}$



In [ ]:
LEARNING_RATE = 0.1
EPOCHS = 1000

b_vector = np.zeros(X_with_intercept.shape[1]) ## initilize intercept and coefficents as 0
cols = ["intercept","log_PFOS", "age", "gender", "BMI"]


def compute_sigmoid(B: np.ndarray, X: np.ndarray):

    # return 1 / (1 + np.exp(-np.dot(X, B))) ## ALT METHOD: np.dot(X, B) computes the dot product between each row of X and B
    return 1/(  1 + np.exp( - X @ B )  ) ## X @ B computes the dot product between each row of X and B


def compute_cost(B: np.ndarray, X: np.ndarray, Y: np.ndarray):
    num_rows = Y.shape[0]
    sigmoid = compute_sigmoid(B, X)

    return np.sum((Y * np.log(1/sigmoid)) + (1 - Y) * np.log(1/sigmoid)) / -num_rows

def partial_derivative(B: np.ndarray, X: np.ndarray, Y: np.ndarray):
    num_rows = Y.shape[0]
    sigmoid = compute_sigmoid(B, X)

    # return (1 / num_rows) * np.dot(X.T, (sigmoid - Y)) ## ALT METHOD
    
    ## (sigmoid - Y)[:,np.newaxis] changes its shape from (n,) to (n, 1)
    return np.sum( (sigmoid - Y)[:,np.newaxis] * X, axis = 0 ) / num_rows

for epoch in range(EPOCHS):

    cost = compute_cost(b_vector, X_with_intercept, Y)
    
    b_vector -= LEARNING_RATE * partial_derivative(b_vector, X_with_intercept, Y)

    b_dict = {col : b for col, b in zip(cols, b_vector)}

    if epoch % 100 == 0 or epoch+1 == 1000:
        print(f"------------------------------------epoch: {epoch}------------------------------------")
        print(f"\t\t\t\tcost: {cost}\nb_dict: {b_dict}")


print("final values: ")
for key, value in b_dict.items(): print(f"{key}: {round(value,8)}")


------------------------------------epoch: 0------------------------------------
				cost: -0.6931471805599453
b_dict: {'intercept': 0.003666666666666667, 'log_PFOS': 9.976846641748102e-05, 'age': 0.005614781879612744, 'gender': 0.0071261720081734915, 'BMI': 0.009237937363079844}
------------------------------------epoch: 100------------------------------------
				cost: -0.6539572177431865
b_dict: {'intercept': 0.13972505594387125, 'log_PFOS': 0.02367971118170717, 'age': 0.2138870505194863, 'gender': 0.2507497017915418, 'BMI': 0.3237468146185928}
------------------------------------epoch: 200------------------------------------
				cost: -0.6522490102240123
b_dict: {'intercept': 0.15402810805273662, 'log_PFOS': 0.03190053668344025, 'age': 0.23758336393223675, 'gender': 0.2717541566678402, 'BMI': 0.3495744796820144}
------------------------------------epoch: 300------------------------------------
				cost: -0.6520341102384797
b_dict: {'intercept': 0.15567719833985094, 'log_PFOS': 0.03

(c) 
> (10 points) Apply your own algorithm to the standardized data and
provide the values of the learned θ.

In [ ]:
## for each row (sample) in x, multiply the variables (x1-x4) by the weights in b
## sum the resulting vector 
## this is the dot product 

print("final intercept and coefficents: ")
for key, value in b_dict.items(): print(f"{key}: {round(value,8)}")

y_predicted = np.dot( X_with_intercept, b_vector )
# y_predicted = np.dot( X, b_vector[1: len(b_vector)] ) + b_vector[0] ## b_vector[0] is the intercept

print("\npredicted y values:\n", y_predicted )


final intercept and coefficents: 
intercept: 0.15589359
log_PFOS: 0.03370199
age: 0.24116572
gender: 0.27433772
BMI: 0.35218456

predicted y values:
 [-0.63768188 -0.35568051  0.28935379 -0.71980975 -0.22064997  1.35101711
 -0.93051684 -0.18920002 -0.26944778  0.31042177  0.28683218 -0.02217559
  0.98501291  0.72088543 -0.21933665 -0.85168726  0.99216161  1.02343907
  1.0547744  -0.26340074  0.41901037  0.22533793  0.01916788  0.2273552
 -0.87785937  0.3756261   0.6163954  -0.42789026 -0.09631984 -0.83017099
 -0.67437333 -0.50946058 -0.20149582 -0.24332532  0.12917514  0.40945855
 -0.14580022  0.90564431  0.21993656  0.39186022 -0.513669   -0.83247607
  0.02116505 -0.12098017  0.74917571 -0.02371777  1.22138296  0.12075989
  0.94379943  0.91592837 -0.0973883   0.45756781  0.79816807 -0.54525029
  0.11661924  0.36770702 -0.7848109  -0.21367765 -0.16292993 -0.06461848
 -1.33210208  1.07282734  0.66071003  0.22931989  1.09987897  0.95323906
  0.63038586  0.1478374  -0.47250965 -0.16040242

In [205]:
diff = np.where(y_predicted > 0, 1, 0) - Y
len(diff[diff > 0]) / 300 ## 78 missclassified

0.26

### Sklearn

(d) 
> (10 points) Apply LogisticRegression in sklearn to the y and the stan-
dardized x. What are the θ values you get from sklearn? Information
about how to apply LogisticRegression in sklear can be found at
https://scikit-learn.org/stable/modules/generated/sklearn.
linear_model.LogisticRegression.html

In [114]:


X_df = df.loc[:,["log_PFOS", "age", "gender", "BMI"]]

Y_df = df["disease"]


logreg = LogisticRegression(random_state=16,  max_iter=1000, penalty=None)

model = logreg.fit(X_df, Y_df)
coef_df = {col : coef for col, coef in zip(list(cols[1:len(cols)]), model.coef_[0])}
# print(coef_df)
# print(X.columns)
print(f"coefficent: {coef_df}\nintercept: {model.intercept_}")



coefficent: {'log_PFOS': 0.033675373184102304, 'age': 0.24156794428388667, 'gender': 0.27431454819455997, 'BMI': 0.35239964201490753}
intercept: [0.1560249]


### statsmodel
(e) 
> (10 points) Add constant to the standardized x using the function
add constant. Instructions about how to use add constant can be
found at:
https://www.statsmodels.org/dev/generated/statsmodels.tools.tools.add_constant.html
Apply Logit in statsmodels to the data with constant 1 added. What
θ do you get? Instructions about how to use statsmodels to do logistic
regression can be found at:
https://www.statsmodels.org/stable/generated/statsmodels.formula.api.logit.html

In [200]:
X_df_with_intercept = statsmodels.tools.tools.add_constant(X_df, prepend=True)
model = api.logit(formula="disease ~ log_PFOS + age + gender + BMI", data = pd.concat([X_df_with_intercept, Y_df], axis = 1))

model.fit().summary()


Optimization terminated successfully.
         Current function value: 0.658576
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                disease   No. Observations:                  300
Model:                          Logit   Df Residuals:                      295
Method:                           MLE   Df Model:                            4
Date:                Sun, 28 Sep 2025   Pseudo R-squ.:                 0.04617
Time:                        12:49:18   Log-Likelihood:                -197.57
converged:                       True   LL-Null:                       -207.14
Covariance Type:            nonrobust   LLR p-value:                 0.0007418
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1559      0.120      1.303      0.193      -0.079       0.390
log_PFOS       0.0337      0.121      0.280      0.780      -0.203       0.270
age            0.2412      0.123      1.963      0.050       0.000       0.482
gender         0.2743      0.120      2.280      0.023       0.038       0.510
BMI            0.3522      0.123      2.868      0.004       0.112       0.593
==============================================================================
"""